In [18]:
import torch
hw = torch.zeros((3, 2, 300, 400))
c2v = torch.ones((3, 8, 300, 400)) * 4
v2c = torch.zeros((3, 8, 300, 400)) * (-3)
diff = (c2v - v2c) / 2

In [20]:
hw[:, 0, ...] = (- diff[:, 0, ...] + diff[:, 2, ...] + diff[:, 4, ...] - diff[:, 6, ...]) /2
hw[:, 1, ...] = (- diff[:, 1, ...] - diff[:, 3, ...] + diff[:, 5, ...] - diff[:, 7, ...]) /2


In [22]:
hw.shape

torch.Size([3, 2, 300, 400])

In [1]:
import os.path as osp
from sys import argv

import mmcv
import wandb
from mmcv import Config
from mmdet.apis import set_random_seed, train_detector

# Let's take a look at the dataset image
from mmdet.datasets import build_dataset
from mmdet.models import build_detector

wandb.login()

cfg = Config.fromfile(f"/home/aiarhipov/centernet/exps/20_cycle_l1/config.py")


set_random_seed(0, deterministic=False)

# Build dataset
datasets = [build_dataset(cfg.data.train), build_dataset(cfg.data.val_loss)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: archiealexarkhipov (centernet). Use `wandb login --relogin` to force relogin


loading annotations into memory...
Done (t=13.36s)
creating index...
index created!
loading annotations into memory...
Done (t=3.86s)
creating index...
index created!


2023-05-06 18:18:22,141 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.


loading annotations into memory...
Done (t=5.15s)
creating index...


2023-05-06 18:18:27,854 - mmdet - INFO - Start running, host: aiarhipov@lorien.atp-fivt.org, work_dir: /home/aiarhipov/centernet/exps/20_cycle_l1_avg_factor_2
2023-05-06 18:18:27,855 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train

index created!


2023-05-06 18:18:32,675 - mmdet - WARNING - No meta information found in the runner. 
wandb:   32 of 32 files downloaded.  
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
2023-05-06 18:31:34,867 - mmdet - INFO - Epoch [1][1000/1372]	lr: 1.249e-03, eta: 1 day, 19:44:10, time: 0.769, data_time: 0.015, memory: 2479, loss_center_heatmap: 8.5272, loss_wh: 2.2763, loss_offset: 0.3174, loss_c2v: 32.6969, loss_v2c: 17.7616, loss: 61.5793, grad_norm: 253.3498


[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 3611/3611, 9.9 task/s, elapsed: 365s, ETA:     0s

2023-05-06 18:48:03,479 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=56.10s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


In [6]:
%debug

> /home/aiarhipov/mmdetection/mmdet/models/losses/utils.py(44)weight_reduce_loss()
     42     # if weight is specified, apply element-wise weight
     43     if weight is not None:
---> 44         loss = loss * weight
     45 
     46     # if avg_factor is not specified, just reduce the loss

torch.Size([8, 8, 128, 128])
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
         